<a href="https://colab.research.google.com/github/acmilannesta/Bert-embedding/blob/master/GoogleQA_tfhub.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Wed Dec 25 23:45:11 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.44       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# bert uncased base
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbu
!mkdir bert_base_uncased
!tar -xzf bbu -C bert_base_uncased

# bert cased base
!wget https://tfhub.dev/tensorflow/bert_en_cased_L-12_H-768_A-12/1?tf-hub-format=compressed -O bbc
!mkdir bert_base_cased
!tar -xzf bbc -C bert_base_cased

# albert base
!wget https://tfhub.dev/google/albert_base/1?tf-hub-format=compressed -O ab
!mkdir albert_base
!tar -xzf ab -C albert_base

# bert uncased large
!wget https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1?tf-hub-format=compressed -O blu
!mkdir bert_large_uncased
!tar -xzf blu -C bert_large_uncased

--2019-12-25 23:45:20--  https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/1?tf-hub-format=compressed
Resolving tfhub.dev (tfhub.dev)... 64.233.189.139, 64.233.189.101, 64.233.189.113, ...
Connecting to tfhub.dev (tfhub.dev)|64.233.189.139|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz [following]
--2019-12-25 23:45:20--  https://storage.googleapis.com/tfhub-modules/tensorflow/bert_en_uncased_L-12_H-768_A-12/1.tar.gz
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.204.128, 2404:6800:4008:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 405800905 (387M) [application/x-tar]
Saving to: ‘bbu’

bbu                 100%[===================>] 387.00M  91.1MB/s    in 5.2s    

2019-12-25 23:45:26 (74.5 MB/s) - ‘bbu’ saved [40580

In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
%tensorflow_version 2.x
import sys
sys.path.insert(1, '/content/drive/My Drive/GoogleQA')
import numpy as np
import pandas as pd
import os, gc
import codecs
import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras.backend import set_session
# from keras.callbacks import Callback
# from keras.models import Model, load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, RepeatedStratifiedKFold, GroupKFold, KFold, RepeatedKFold
from scipy.stats import spearmanr
from tqdm import tqdm
import tensorflow_hub as hub
# from albert_tokenization import FullTokenizer
from tokenizer import Tokenizer
from warmup_v2 import AdamWarmup, calc_train_steps
tf.get_logger().setLevel('ERROR') 
# from keras_bert import load_trained_model_from_checkpoint, Tokenizer, AdamWarmup, calc_train_steps, get_custom_objects
from hyperopt import fmin, hp, tpe, STATUS_OK, Trials
from functools import partial
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0-rc1


In [0]:
data_path = 'drive/My Drive/GoogleQA/Data/'
aux_path = 'drive/My Drive/GoogleQA/Web Scrap/'
train = pd.read_csv(data_path+'train.csv')
test = pd.read_csv(data_path+'test.csv')
sub = pd.read_csv(data_path+'sample_submission.csv')
# qa_aux = pd.read_csv(aux_path+'df_qa_extracted.csv')[['qa_id', 'question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes']]
# quser_aux = pd.read_csv(aux_path+'df_question_user_extracted.csv')
# auser_aux = pd.read_csv(aux_path+'df_answer_user_extracted.csv')
target_col = train.columns.tolist()[11:42]

train['class'] = train.host.apply(lambda x: x.split('.')[0])
train['question_title'] = train['category']+' '+train['class']+' '+train['question_title']#+' '+train['question_body']
test['class'] = test.host.apply(lambda x: x.split('.')[0])
test['question_title'] = test['category']+' '+test['class']+' '+test['question_title']#+' '+test['question_body']
 
# category_encoder = LabelEncoder().fit(pd.concat([train['category'], test['category']]))
# host_encoder = LabelEncoder().fit(pd.concat([train['host'], test['host']]))
# train['category'] = category_encoder.transform(train['category'])
# test['category'] = category_encoder.transform(test['category'])
# train['host'] = host_encoder.transform(train['host'])
# test['host'] = host_encoder.transform(test['host'])




In [0]:
MAXLEN = 512 #@param {type:"slider", min:128, max:512, step:32}
BATCH_SIZE = 4 #@param {type:'slider', min:1, max:32, step:1}
NUM_EPOCHS = 3
NUM_CLASSES = 30
NUM_AUX = 6
LR = 5e-5
MIN_LR = 0
model_path = 'bert_base_uncased' #@param ['bert_base_uncased', 'bert_base_cased', 'bert_large_uncased', 'albert_base']
save_path = 'bbu4'
save_model = 'bbu4' 
CASED = False #@param ['True', 'False'] {type:"raw"}
# OUTPUT_TRAIN = 'train_bert_ipredcv1415_oof.csv'
# OUTPUT_TEST = 'test_bert_large.npy'
# model_path = 'uncased_L-12_H-768_A-12' #@param ['uncased_L-12_H-768_A-12', 'wwm_uncased_L-24_H-1024_A-16', 'uncased_L-24_H-1024_A-16']
# target_q_col = train.columns.tolist()[11:32]
# target_a_col = train.columns.tolist()[32:42]


## Bert tokenizer

In [0]:
def _trim_input(title, question, answer, max_sequence_length=MAXLEN,
                t_max_len=20, q_max_len=246, a_max_len=246):
    # t = tokenizer.tokenize(title)
    # q = tokenizer.tokenize(question)
    # a = tokenizer.tokenize(answer)
    t_len = len(title)
    q_len = len(question)
    a_len = len(answer)
    if (t_len + q_len + a_len) > max_sequence_length:
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + np.floor((t_max_len - t_len) / 2)
            q_max_len = q_max_len + np.ceil((t_max_len - t_len) / 2)
        else:
            t_new_len = t_max_len
        if a_max_len > a_len:
            a_new_len = a_len
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
        if t_new_len + a_new_len + q_new_len!= max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"
                    % (max_sequence_length, (t_new_len + a_new_len + q_new_len)))
        head_t_new_len = int(0.25 * t_new_len)
        tail_t_new_len = int(t_new_len - head_t_new_len)

        head_q_new_len = int(0.25 * q_new_len)
        tail_q_new_len = int(q_new_len - head_q_new_len)

        head_a_new_len = int(0.25 * a_new_len)
        tail_a_new_len = int(a_new_len - head_a_new_len)

        title = title[:head_t_new_len] + title[-tail_t_new_len:]
        question = question[:head_q_new_len] + question[-tail_q_new_len:]
        answer = answer[:head_a_new_len] + answer[-tail_a_new_len:]

    return title, question, answer

In [0]:
token_dict = {}
with codecs.open(os.path.join(model_path, 'assets/vocab.txt'), 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        token_dict[token] = len(token_dict)
tokenizer = Tokenizer(token_dict, cased=CASED)

def convert_data(data_df, branch='training'):
    data_df = data_df.reset_index(drop=True)
    global tokenizer
    global MAXLEN
    global target_col
    ids, segments = [], []
    # q_title, q_body, answer = [], [], []
    for i in tqdm(range(len(data_df))):
        a, _ = tokenizer.encode(data_df.loc[i, 'question_title'])
        b, _ = tokenizer.encode(data_df.loc[i, 'question_body'])
        c, _ = tokenizer.encode(data_df.loc[i, 'answer'])
        # if len(a) + len(b) >512:
        #     if (len(a) > 256) & (len(b) > 256):
        #         a, b = a[:256], b[:256]
        #     elif len(a) > len(b):
        #         a = a[:(512-len(b))]
        #     elif len(a) <= len(b):
        #         b = b[:(512-len(a))]
        a, b, c = _trim_input(a, b, c)
        ids.append(a + b + c)
        segments.append([0] * len(a + b) + [1] * len(c))
    # aux = data_df[['question_viewed_times', 'question_votes', 'answer_counts', 'answer_votes', 'answer_user_reputation', 'question_user_reputation']]
    # aux = aux.apply(lambda x: (x - x.mean()) / x.std())    
    # aux = data_df[['category', 'host']]        
    if branch == 'training':
        targets = data_df[target_col]
        return [ids, segments], np.array(targets)
        # return [ids, segments], np.array(aux, dtype='float32'), np.array(targets)
    else:
        # return [ids, segments], np.array(aux, dtype='float32')
        return [ids, segments]

In [0]:
"""## Data Generator"""
def seq_padding(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([x, [padding] * (ML - len(x))]) if len(x) < ML else x[:ML] for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)
    
def get_masks(X, padding=0):
    L = [len(x) for x in X]
    ML = min(max(L), MAXLEN)
    # ML = MAXLEN
    out = np.array([np.concatenate([[1]*len(x), [padding] * (ML - len(x))]) if len(x) < ML else [1]*ML for x in X])
    return tf.convert_to_tensor(out, dtype=tf.int32)

class data_generator:
    def __init__(self, data, batch_size=BATCH_SIZE, branch='train'):
        self.data = data
        self.batch_size = batch_size
        self.branch = branch
        # self.q_a = q_a
        self.steps = len(self.data) // self.batch_size
        if len(self.data) % self.batch_size != 0:
            self.steps += 1

    def __len__(self):
        return self.steps

    def __iter__(self):
        while True:
            if self.branch == 'train':
                np.random.shuffle(self.data)
            for i in range(self.steps):
                d = self.data[i * self.batch_size: (i + 1) * self.batch_size]
                X1 = seq_padding([x[0] for x in d])
                X2 = get_masks([x[0] for x in d])    
                X3 = seq_padding([x[1] for x in d])
                # X3 = np.zeros_like(X1)
                # aux1 = tf.convert_to_tensor([x[2][:, 0] for x in d], dtype=tf.int32)
                # aux2 = tf.convert_to_tensor([x[2][:, 1] for x in d], dtype=tf.int32)
                if self.branch == 'test':
                    # aux = np.array([x[3] for x in d])
                    yield [X1, X2, X3]
                    # yield [X1, X2, X3, X4, X5, X6]
                else:
                    Y = tf.convert_to_tensor([x[2] for x in d], dtype=tf.float32)
                    # aux = np.array([x[4] for x in d])
                    yield [X1, X2, X3], Y
                    # yield [X1, X2, X3, aux1, aux2], Y


In [0]:
def model_build(len_train, lr=LR, epochs=NUM_EPOCHS, bert_trainable=True):
    global NUM_CLASSES
    global BATCH_SIZE
    global NUM_EPOCHS
    global MIN_LR
    # global LR
    global MAXLEN
    global model_path
    global NUM_AUX
 
    q_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_word_ids")
    q2_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_input_masks")
    q3_in = keras.layers.Input(shape=(None,), dtype=tf.int32, name="q_segment_ids")
    # input_category = keras.layers.Input((1,), dtype=tf.int32, name='input_category')
    # input_host = tf.keras.layers.Input((1,), dtype=tf.int32, name='input_host')

    bert_layer = hub.KerasLayer(model_path, trainable=bert_trainable)
    _, q_inputs  = bert_layer([q_in, q2_in, q3_in])
    q_outputs = keras.layers.GlobalAveragePooling1D()(q_inputs)
    dense = keras.layers.Dropout(0.1)(q_outputs)

    # category_emb = keras.layers.Embedding(input_dim=len(category_encoder.classes_)+1, output_dim=32)(input_category)
    # category_emb = keras.layers.SpatialDropout1D(0.1)(category_emb)

    # host_emb = keras.layers.Embedding(input_dim=len(category_encoder.classes_)+1, output_dim=32)(input_host)
    # host_emb = keras.layers.SpatialDropout1D(0.1)(host_emb)

    # features_dense = keras.layers.concatenate([category_emb, host_emb], axis=1)
    # features_dense = keras.layers.Flatten()(features_dense)

    # dense = keras.layers.concatenate([dense, features_dense])
    outputs = keras.layers.Dense(NUM_CLASSES, activation='sigmoid')(dense)
    # aux_out = keras.layers.Dense(NUM_AUX)(dense)
    model = keras.Model([q_in, q2_in, q3_in], [outputs])

    decay_steps, warmup_steps = calc_train_steps(
        len_train,
        batch_size=BATCH_SIZE,
        epochs=epochs,
    )

    model.compile(
        loss=['binary_crossentropy'],
        # loss_weights = [10, 0.25],
        optimizer=AdamWarmup(
            decay_steps=decay_steps,
            warmup_steps=warmup_steps,
            lr=lr,
            min_lr=MIN_LR,
            ))
    #      optimizer=keras.optimizers.Adam(LR),
    # )

    return model

## train model

In [0]:
# pred = np.zeros((len(test), NUM_CLASSES))
# kf = RepeatedKFold(n_splits=10, n_repeats=1, random_state=1627)
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]

def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, stage=1, rho=-1):
        global NUM_EPOCHS
        global save_model
        super(keras.callbacks.Callback, self).__init__()
        self.stage = stage
        self.rho = rho
        # self.rho = rho
    def on_epoch_end(self, epoch, logs={}):
        val_pred = self.model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
        score = compute_spearmanr(val_y, val_pred)
        print('Spearman - {:.5f}'.format(score))
        # if self.stage==1:
        if score > self.rho:
            self.rho = score
            print('--Save Model--')
            self.model.save('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-{:}.h5'.format(save_path, save_model, i, self.stage))
        # elif self.stage==2:
        #     if score > self.rho:
        #         self.rho = score
        #         print('--Save Model--')
        #         self.model.save('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-3.h5'.format('bbu4', 'bbu4', i))
        #     else:
        #         self.model.stop_training = True


In [0]:
for i, (tr_idx, val_idx) in enumerate(idx[4:5], 5):
    keras.backend.clear_session()
    gc.collect()

    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]
    # tr_aux, val_aux = train_aux.loc[tr_idx], train_aux.loc[val_idx]
    tr_x, tr_y = convert_data(tr)
    val_x, val_y = convert_data(val)
    train_D = data_generator(list(zip(tr_x[0], tr_x[1], tr_y)))
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_y)), branch='valid')
    
    # stage 1 fine tunning
    # ieval = IntervalEval()
    # model = model_build(len(tr))
    # model.fit_generator(
    #     train_D.__iter__(),
    #     steps_per_epoch=len(train_D),
    #     epochs=NUM_EPOCHS,
    #     callbacks = [ieval]
    # )   

    # stage 2 fine tunning
    ieval = IntervalEval(stage=2, rho=0.39059)
    model = model_build(len(tr), lr=1e-3, epochs=4, bert_trainable=False)
    model.load_weights('drive/My Drive/GoogleQA/Models/{:}/{:}-{:}-1.h5'.format(save_path, save_model, i))

    # test_D = data_generator(list(zip(test_x[0], test_x[1])), branch='test')
    
    model.fit_generator(
        train_D.__iter__(),
        steps_per_epoch=len(train_D),
        epochs=4,
        callbacks = [ieval]
    )

    gc.collect()

In [0]:
# oof-prediction
model = model_build(len(tr), bert_trainable=False)
oof_pred = train[['qa_id']+target_col].copy()
oof_pred[target_col] = 0

for i, (tr_idx, val_idx) in enumerate(idx[1:], 2):
    print('\nFold - {:}\n'.format(i))
    tr, val = train.loc[tr_idx], train.loc[val_idx]

    val_x, val_y = convert_data(val)
    valid_D = data_generator(list(zip(val_x[0], val_x[1], val_y)), branch='valid')

    model.load_weights('drive/My Drive/GoogleQA/Models/{}/{}-{}-2.h5'.format(save_path, save_model, i))
    val_pred = model.predict_generator(valid_D.__iter__(), len(valid_D), verbose=1)
    oof_pred.loc[val_idx, target_col] += val_pred /2

    
    score = compute_spearmanr(val_y, val_pred)
    print('Spearman - {:.5f}'.format(score))

    gc.collect()

oof_pred.to_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu4.csv', index=False)

## oof-prediction hyperparameterize
1. Number of bins
2. Cutoffs

### Just use "90" transform

In [33]:
# bbu3: 0.42130
# bbu4: 0.43406
oof_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/train_oof_pred_bbu4.csv')
score = compute_spearmanr(np.floor(oof_pred[target_col].values*90)/90, train[target_col].values)
print('Total Score: {:.5f}'.format(score))

# just using "90" transformation
scores = []
for col, col_trues, col_pred in zip(target_col, train[target_col].values.T, (np.floor(oof_pred[target_col].values *90)/90).T):
    corr = spearmanr(col_trues, col_pred).correlation
    print('{} - {:.5f}'.format(col, corr))
    scores.append(corr)

hyper1 = pd.DataFrame(dict(col=target_col, score1=scores))

Total Score: 0.43406


### Use lower (to 0) resetting

In [35]:
# Just use zero
corrparams = {
    'lower': hp.randint('lower', len(oof_pred)),
    # 'upper': hp.randint('upper', len(oof_pred))
    }
def f_min(params, col):
    tmp = oof_pred.copy()[col].values
    loweridx = tmp.argsort()[:params['lower']]
    # upperidx = tmp.argsort()[-params['upper']:]
    tmp[loweridx] = 0
    # tmp[upperidx] = 1
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

score, lower = [], []
for col in target_col:
    f = partial(f_min, col=col)
    trials = Trials()
    best = fmin(f, corrparams, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {} - lower: {:.3f}'.format(
        col, 
        -trials.best_trial['result']['loss'],
        best['lower']/len(oof_pred),
        # best['upper']/len(oof_pred)
        ))
    score.append(-trials.best_trial['result']['loss'])
    lower.append(round(best['lower']/len(oof_pred), 2))
# bbu3: 0.44169
# bbu4: 0.45214
print(np.mean(score))

hyper2 = pd.DataFrame(dict(col = target_col, lower = lower, score2 = score))

,col,lower,score2
0,question_asker_intent_understanding,0.07,0.37807
1,question_body_critical,0.00,0.63432
2,question_conversational,0.88,0.52646
3,question_expect_short_answer,0.00,0.30577
4,question_fact_seeking,0.00,0.37506
5,question_has_commonly_accepted_answer,0.00,0.43002
6,question_interestingness_others,0.46,0.35939
7,question_interestingness_self,0.62,0.52263
8,question_multi_intent,0.54,0.61034
9,question_not_really_a_question,0.93,0.13796


### Use n-binning

In [0]:
# bbu3: 0.43042
# bbu4: 

def f1_min(params, col):
    tmp = oof_pred.copy()[col].values
    paramlist = [params['p'+str(i)] for i in range(len(params))]
    bins = np.percentile(tmp, paramlist)
    tmp = np.digitize(tmp, np.sort(bins)) / len(bins)
    score = spearmanr(train[col].values, tmp).correlation
    return {'loss': -round(score, 5), 'status': STATUS_OK}

def f2_min(params, col):
    cutoff = {'p'+str(i): hp.randint('p'+str(i), 101) for i in range(params['num_bin'])}
    f = partial(f1_min, col=col)
    trials = Trials()
    best = fmin(f, cutoff, tpe.suggest, 100, rstate=np.random.RandomState(0), trials=trials, 
                show_progressbar=False)
    # print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score = -trials.best_trial['result']['loss']
    return {'loss': -round(score, 5), 'status': STATUS_OK, 'cutoff': list(best.values())}

score, cutoffs = [], []
for col in target_col:
    numbin = {'num_bin': hp.randint('num_bin', 19)+2}
    f = partial(f2_min, col=col)
    trials = Trials()
    best = fmin(f, numbin, tpe.suggest, 10, rstate=np.random.RandomState(0), trials=trials, show_progressbar=False)
    print('{} - {}'.format(col, -trials.best_trial['result']['loss']))
    score.append(-trials.best_trial['result']['loss'])
    cutoffs.append(trials.best_trial['result']['cutoff'])

print(np.mean(score))
hyper3 = pd.DataFrame(dict(col = target_col, cutoffs_pct = cutoffs, score3 = score))


question_asker_intent_understanding - 0.37851
question_body_critical - 0.63356
question_conversational - 0.5164
question_expect_short_answer - 0.30806
question_fact_seeking - 0.37662
question_has_commonly_accepted_answer - 0.48123
question_interestingness_others - 0.35821
question_interestingness_self - 0.51695
question_multi_intent - 0.60606
question_not_really_a_question - 0.13406
question_opinion_seeking - 0.48651
question_type_choice - 0.76248
question_type_compare - 0.53144
question_type_consequence - 0.27764
question_type_definition - 0.6253
question_type_entity - 0.59428
question_type_instructions - 0.79094


In [0]:
question_asker_intent_understanding - 0.37912
question_body_critical - 0.63471
question_conversational - 0.47301
question_expect_short_answer - 0.30801
question_fact_seeking - 0.37662
question_has_commonly_accepted_answer - 0.48017
question_interestingness_others - 0.35821

In [73]:
# 0.44462
hyper = hyper1.merge(hyper2, on='col').merge(hyper3, on='col')
hyper.to_csv('drive/My Drive/GoogleQA/Data/hyper.csv', index=False)
hyper['score'] = hyper[['score1', 'score2', 'score3']].max(axis=1)
hyper['score'].mean()

,col,lower,score1,cutoffs_pct,score2,score
0,question_asker_intent_understanding,0.00,0.37869,"[83, 43, 61, 72, 56, 91, 6, 4, 73, 71, 98, 96,...",0.38050,0.38050
1,question_body_critical,0.00,0.62999,"[32, 55, 81, 86, 54, 79, 42, 71, 3, 84, 69, 41...",0.63132,0.63132
2,question_conversational,0.91,0.50808,"[75, 100, 92, 85, 74, 83]",0.48317,0.50808
3,question_expect_short_answer,0.00,0.30653,"[81, 14, 1, 83, 5, 91, 79, 68, 16, 58, 88, 56,...",0.30914,0.30914
4,question_fact_seeking,0.00,0.37795,"[89, 11, 71, 15, 63, 14, 97, 35, 0, 16, 88, 55...",0.37937,0.37937
5,question_has_commonly_accepted_answer,0.00,0.43624,"[29, 7, 17]",0.48810,0.48810
6,question_interestingness_others,0.24,0.34862,"[85, 63, 3, 3, 97, 55, 63, 78, 49, 22, 36, 91,...",0.34866,0.34866
7,question_interestingness_self,0.53,0.50938,"[80, 100, 93, 75, 83, 73, 95, 97, 53]",0.50707,0.50938
8,question_multi_intent,0.40,0.58301,"[81, 76, 61, 66, 92, 56, 40]",0.58103,0.58301
9,question_not_really_a_question,0.88,0.10707,"[1, 100, 88]",0.10102,0.10707


## test prediction

In [0]:
test_pred = pd.read_csv('drive/My Drive/GoogleQA/Data/test_oof_pred.csv')
hyper = pd.read_csv('drive/My Drive/GoogleQA/Data/hyper.csv')
pred = test_pred[target_col].copy().values

,col,pct
0,question_conversational,0.91
1,question_interestingness_others,0.24
2,question_interestingness_self,0.53
3,question_multi_intent,0.40
4,question_not_really_a_question,0.88
5,question_opinion_seeking,0.04
6,question_type_choice,0.48
7,question_type_compare,0.95
8,question_type_consequence,0.99
9,question_type_definition,0.93


In [0]:
for col in [x for x in target_col if x not in hyper.col.tolist()]:
    pred[:, target_col.index(col)] = (pred[:, target_col.index(col)]//(1/90))/90
for col in hyper.col.tolist():
    if hyper.loc[hyper.col==col, 'pct'].values == 1:
        pct = hyper.loc[hyper.col==col, 'pct'] - 0.01
    else:
        pct = hyper.loc[hyper.col==col, 'pct']
    changerow = int(len(test) * pct)
    rowidx = pred[:, target_col.index(col)].argsort()[:changerow]
    pred[rowidx, target_col.index(col)] = 0



19    0.17
Name: pct, dtype: float64

In [0]:
for i in range(pred.shape[1]):
    print(i, len(np.unique(pred[:, i])))

0 24
1 53
2 44
3 58
4 49
5 54
6 363
7 225
8 287
9 59
10 458
11 249
12 25
13 6
14 35
15 59
16 354
17 444
18 382
19 6
20 435
21 16
22 25
23 8
24 9
25 24
26 363
27 458
28 439
29 397


## USE Model

In [0]:

# train['sentence'] = train['question'] + ' ' + train['answer']
embed1 = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")
# embed2 = hub.load('https://tfhub.dev/google/nnlm-en-dim128/2')

def UniversalEmbedding(x):
    return embed1(tf.squeeze(tf.cast(x, tf.string)))

# def NNLMEmbedding(x):
#     return embed2(tf.squeeze(tf.cast(x, tf.string)))

def use_model():
    
    q = keras.layers.Input(shape=(1,), dtype=tf.string)
    qb = keras.layers.Input(shape=(1,), dtype=tf.string)
    a = keras.layers.Input(shape=(1,), dtype=tf.string)

    def hidden_layer(input_layer):
        x = keras.layers.Lambda(UniversalEmbedding, output_shape=(512, ))(input_layer)
        # embed2 = keras.layers.Lambda(NNLMEmbedding, output_shape=(128, ))(input_layer)
        # embed = keras.layers.concatenate([embed1, embed2])
        x = keras.layers.Dropout(0.2)(x)
        x = keras.backend.expand_dims(x, axis=1)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # x = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(128, return_sequences=True))(x)
        # GP = keras.layers.GlobalMaxPooling1D()(x)
        # GP = keras.layers.Dropout(0.2)(GP)
        # AP = keras.layers.GlobalAveragePooling1D()(x)
        # AP = keras.layers.Dropout(0.2)(AP)
        # hidden = keras.layers.concatenate([GP, AP])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])
        # hidden = keras.layers.add([hidden, keras.layers.Dense(512, activation='relu')(hidden)])

        x1 = keras.layers.Bidirectional(keras.layers.CuDNNLSTM(256, return_sequences=True))(x)
        x2 = keras.layers.Bidirectional(keras.layers.CuDNNGRU(128, return_sequences=True))(x1)
        max_pool1 = keras.layers.GlobalMaxPooling1D()(x1)
        max_pool2 = keras.layers.GlobalMaxPooling1D()(x2)
        hidden = keras.layers.concatenate([max_pool1, max_pool2])
        return hidden

    q_embed, qb_embed, a_embed = hidden_layer(q), hidden_layer(qb), hidden_layer(a)
    all_embed = keras.layers.concatenate([q_embed, qb_embed, a_embed])
    dense = keras.layers.Dense(256, activation='relu')(all_embed)
    pred = keras.layers.Dense(30, activation='sigmoid')(dense)
    model = keras.Model(inputs=[q, qb, a], outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=5e-3))
    # del embed
    # gc.collect()
    return model


In [0]:
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=0)
idx = [x for x in kf.split(train, groups=train.question_body)]
def compute_spearmanr(trues, preds):
    rhos = []
    for col_trues, col_pred in zip(trues.T, preds.T):
        rhos.append(spearmanr(col_trues, col_pred).correlation)
    return np.mean(rhos)

class IntervalEval(keras.callbacks.Callback):
    def __init__(self, val_data, label):
        super(keras.callbacks.Callback, self).__init__()
        self.val_data = val_data
        self.score = 0
        self.maxscore = 0
        self.label = label
        self.patience = 3
        self.count = 0
    def on_epoch_end(self, epoch, logs={}):
        # if epoch == 3:
        val_pred = self.model.predict(self.val_data, batch_size=16, verbose=1)
        score = compute_spearmanr(self.label, val_pred)
        print('Spearman - {:.5f}'.format(score))
        if self.maxscore>=0.3:
            self.patience=2
        if (score < self.score):
            self.score = score
            self.count+=1
            if self.count==self.patience:
                self.model.stop_training=True
        elif score > self.maxscore:
            self.score = score
            self.maxscore = score
            self.count = 0
            if self.maxscore > 0.3:
                self.model.save('drive/My Drive/GoogleQA/Models/{:}/use-model-{:}.h5'.format(save_model, i))
        else:
            self.score = score
            self.count = 0
          

In [0]:
with tf.Session() as session:
    keras.backend.set_session(session)
    session.run(tf.global_variables_initializer())  
    session.run(tf.tables_initializer())
    for i, (tr_idx, val_idx) in enumerate(idx, 1):
        print('\nFold - {:}\n'.format(i))
        tr, val = train.loc[tr_idx], train.loc[val_idx]
        tr_x = [tr['question_title'], tr['question_body'], tr['answer']]
        val_x = [val['question_title'], val['question_body'], val['answer']]
        ieval = IntervalEval(val_data=val_x, label=val[target_col].values)
        model = use_model()
        history = model.fit(
            tr_x,
            tr[target_col],
            epochs=100, 
            batch_size=16,
            callbacks=[ieval])
        del model, tr, val, tr_x, val_x
        gc.collect()

# with tf.Session() as session:
#     keras.backend.set_session(session)
#     session.run(tf.global_variables_initializer())  
#     session.run(tf.tables_initializer())
#     model.load_weights('use-model-1.h5')
#     preds = model.predict(val['sentence'], batch_size=16, verbose=1)
#     score=compute_spearmanr(val[target_col].values, preds)
#     print('Spearman - {:.5f}'.format(score))


Fold - 1

Train on 4863 samples
Epoch 1/100
1216/1216 [==============================] - 33s 27ms/sample
Spearman - 0.35849
4863/4863 [==============================] - 170s 35ms/sample - loss: 0.3946
Epoch 2/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.36453
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3720
Epoch 3/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37807
4863/4863 [==============================] - 85s 18ms/sample - loss: 0.3628
Epoch 4/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37832
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3549
Epoch 5/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37996
4863/4863 [==============================] - 86s 18ms/sample - loss: 0.3462
Epoch 6/100
1216/1216 [==============================] - 16s 13ms/sample
Spearman - 0.37266
4863/4863 [=======================